### Imports

In [1]:
from z3 import * # must be at least version 4.12.2. 
import pandas as pd
import numpy as np
import operator
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from time import time
from tqdm import tqdm
from tabulate import tabulate
from itertools import product

# Class for a "functional" dictionary, which can be also be accessed with round brackets. 
class fdict(dict):
    def __call__(self, k):
        return self[k]

### Load the dataframe, initialize some variables.

In [2]:
# Load the dataframe.
datasets = [
    # 'churn.csv',
    # 'compas.csv',
    # 'compas-small.csv',
    # 'admission.csv',
    # 'corels.csv',
    # 'mushrooms.csv',
    # 'tort.csv',
    # 'welfare.csv',
    # 'welfare-simplified.csv',
    'welfare-jair.csv',
]

# if datasets[0].startswith('welfare'):

ds = datasets[0]
csv = "data/" + ds
df = pd.read_csv(csv)

# Shuffle the data.
df = df.sample(frac=1).reset_index(drop=True)

# Limit df size.
df = df[:1000]

# Create the set of dimension names. 
D = df.columns.values

# Check whether the data contains a 'label' column.
assert "Label" in df.columns.values, "There is no 'Label' column in the csv data. Make sure the data contains a 'Label' column with rational values."

# Some configuration variables. 
verb = True
method='logreg'
# method='pearson'

# Print some information about the dataset.
print("Some statistics about the dataset:")
print(df.describe())
print()

# Convert columns with values only in {0, 1} to type 'bool'. 
for d in D[:-1]:
    if df.dtypes[d] == 'int64' and df[d].min() == 0 and df[d].max() == 1:
        df[d] = df[d].replace(0, "F").replace(1, "T")

# Print a small sample of the dataset.
print("The first 5 rows of the dataframe:")
print(df.head())
print()

# Print column datatype information. 
print("Column datatypes:")
for d in df.columns.values:
    print(f"{d}: {df.dtypes[d]}")
print()

# Initialize the solver. 
s = Solver()

Some statistics about the dataset:
               age    is_spouse    is_absent  capital_resources
count  1000.000000  1000.000000  1000.000000        1000.000000
mean     70.409000     0.806000     0.178000        2539.808000
std      24.835066     0.395627     0.382704        2329.830681
min       0.000000     0.000000     0.000000           1.000000
25%      64.000000     1.000000     0.000000         979.000000
50%      75.000000     1.000000     0.000000        1908.000000
75%      88.250000     1.000000     0.000000        2842.250000
max     100.000000     1.000000     1.000000        9953.000000

The first 5 rows of the dataframe:
   age paid_contribution_1 paid_contribution_2 paid_contribution_3   
0   98                  no                  no                  no  \
1   81                 yes                 yes                  no   
2   83                 yes                 yes                 yes   
3   99                 yes                 yes                 yes   
4  

### Determine the appropriate sorts for the dimensions.

In [3]:
# A dictionary mapping a dimension to its Z3 sort.
sorts = {}

# A dictionary mapping a dimension value to its Z3 value.
cast = {}

# Loop over the dimensions and fill the sorts and cast dicts.
for d in df.columns.values:
    if df.dtypes[d] == 'float64':
        sorts[d] = RealSort()
        cast[d] = RealVal
    else:
        sorts[d] = IntSort()
        # cast[d] = IntVal # This should work but doesn't due to a bug in Z3.
        cast[d] = lambda x: IntVal(int(x)) # wrap IntVal in a python integer cast to avoid the bug.

# Make a variable fact siutation.
x = {d : Const(f'x_{d}', sorts[d]) for d in D[:-1]}

### Determine the dimensions orders. 

In [4]:
# Initialize the output variables.
orders = {}
A = []
tl = {}
coeffs = None

# Determine the order based on a method that works with a 'coefficient function'.
if method == 'linreg':
    assert False, "Method not (yet) implemented."

if method == 'pearson':
    coeffs = pd.get_dummies(df).corr()["Label"]
    
# Create the coefficient dictionary using logistic regression.
elif method == 'logreg':
    # Create dummy variables for the categorical dimensions.
    ddf = pd.get_dummies(df.drop("Label", axis='columns'))

    # Fit the logistic regression model.
    X = ddf.to_numpy()
    X = preprocessing.StandardScaler().fit(X).transform(X)
    y = df["Label"].to_numpy()
    clf = LogisticRegression(random_state=0).fit(X, y)

    # Create a dictionary mapping the dimensions to their logreg coefficients.
    coeffs = dict(zip(ddf.columns.values, clf.coef_[0]))

# Determine the dimension orders on the basis of the coefficients.
for d in D[:-1]:
    match df.dtypes[d]:
        # Case for numerical dimensions.
        case 'int64' | 'float64':
            # Assign the dimen order based on the coefficient.
            orders[d] = operator.le if coeffs[d] > 0 else operator.ge

            # Print the resulting order, if enabled. 
            if verb:
                print(f"{d}: {'Ascending' if coeffs[d] > 0 else 'Descending'} ({round(coeffs[d], 2)})")

        # Case for categorical dimensions.
        case 'object' | 'bool':
            # Sort the values according to their coefficient.
            dvals = df[d].unique()
            sdvals = sorted(dvals, key=lambda x: coeffs[f'{d}_{x}'])
            orders[d] = operator.le

            # Create a dict for translating the old value names to the new ones. 
            cast[d] = fdict(zip(dvals, [sdvals.index(val) for val in dvals]))
            
            # Add a boundary condition. 
            s.add(And(x[d] >= 0, x[d] <= max(cast[d].values())))

            # Print the resulting order, if enabled. 
            if verb:
                print(f"{d}: " + " < ".join([f"[{cast[d](v)}] {v} ({round(coeffs[f'{d}_{v}'], 2)})" for v in sdvals]))

# # Try all possible welfare-simp orders. 
# print("Logreg orders:")
# print(orders)
# i = 15
# ops = [operator.le, operator.ge]
# porders = list(product(ops, repeat=4))
# orders = dict(zip(D[:-1], porders[i]))
# print(f"Order number {i}:")
# print(orders)

# Add additional axioms and create a labelling formula, 
# depending on the choice of dataset. 
P = None
match ds:
    case 'corels.csv':
        A += [
            x['Age'] >= 18, 
            x['Priors'] >= 0
        ]
        C1 = And(x['Age'] >= 18, x['Age'] <= 20)
        C2 = And(x['Age'] >= 21, x['Age'] <= 23, x['Priors'] >= 2, x['Priors'] <= 3)
        C3 = x['Priors'] > 3
        P = Or([C1, C2, C3])
    case 'tort.csv':
        C1 = x['cau'] == cast['cau']('T')
        C2 = Or(x['ico'] == cast['ico']('T'), x['ila'] == cast['ila']('T'), x['ift'] == cast['ift']('T'))
        C3 = Or(x['vun'] == cast['vun']('T'), And(x['vst'] == cast['vst']('T'), x['jus'] == cast['jus']('F')), And(x['vrt'] == cast['vrt']('T'), x['jus'] == cast['jus']('F')))
        C4 = x['dmg'] == cast['dmg']('T')
        C5 = Or(x['vst'] == cast['vst']('F'), x['prp'] == cast['prp']('T'))
        P = And([C1, C2, C3, C4, C5])
    case 'welfare.csv':
        A += [
            x['age'] >= 0,
            x['age'] <= 100,
            x['distance_to_hospital'] >= 0,
            x['distance_to_hospital'] <= 100,
            x['capital_resources'] >= 0,
            x['capital_resources'] <= 10000,
        ]
        C1 = Or(And(x['gender'] == cast['gender']('f'), x['age'] >= 60), And(x['gender'] == cast['gender']('m'), x['age'] >= 65))
        con = [f'paid_contribution_{i}' for i in range(1, 6)]
        C2 = Or([And([x[d] == cast[d]('yes') for d in con[:i] + con[i+1:]]) for i in range(5)])
        C3 = x['is_spouse'] == cast['is_spouse']('T')
        C4 = x['is_absent'] == cast['is_absent']('F')
        C5 = Not(x['capital_resources'] >= 3000)
        C6 = Or(And(x['patient_type'] == cast['patient_type']('in'),  x['distance_to_hospital'] < 50),
                And(x['patient_type'] == cast['patient_type']('out'), x['distance_to_hospital'] >= 50))
        P = And([C1, C2, C3, C4, C5, C6])
    case 'welfare-simplified.csv':
        A += [
            x['age'] >= 0,
            x['age'] <= 100,
            x['distance_to_hospital'] >= 0,
            x['distance_to_hospital'] <= 100,
        ]
        C1 = Or(And(x['gender'] == cast['gender']('f'), x['age'] >= 60), And(x['gender'] == cast['gender']('m'), x['age'] >= 65))
        C6 = Or(And(x['patient_type'] == cast['patient_type']('in'),  x['distance_to_hospital'] < 50),
                And(x['patient_type'] == cast['patient_type']('out'), x['distance_to_hospital'] >= 50))
        P = And([C1, C6])
    case 'welfare-jair.csv':
        A += [
            x['age'] >= 0,
            x['age'] <= 100,
            x['capital_resources'] >= 0,
            x['capital_resources'] <= 10000,
        ]
        C1 = x['age'] >= 60
        con = [f'paid_contribution_{i}' for i in range(1, 6)]
        C2 = Or([And([x[d] == cast[d]('yes') for d in con[:i] + con[i+1:]]) for i in range(5)])
        C3 = x['is_spouse'] == cast['is_spouse']('T')
        C4 = x['is_absent'] == cast['is_absent']('F')
        C5 = Not(x['capital_resources'] >= 3000)
        P = And([C1, C2, C3, C4, C5])

# Add the relevant axioms to the solver. 
s.add(A)
assert s.check() == sat

def eq(a, x):
    return And([x[d] == a[d] for d in D[:-1]])

def f(a, x):
    if a['Label'] == True:
        return And([orders[d](a[d], x[d]) for d in D[:-1]])
    else:
        return And([orders[d](x[d], a[d]) for d in D[:-1]])

def F(CB, x):
    return Or([f(a, x) for a in CB])

def g(a, x):
    return And(f(a, x), Not(eq(a, x)))

def G(CB, x):
    return Or([g(a, x) for a in CB])

# Create the case base and its (strict) forcing formulae.
CB = {o : [{d : cast[d](r[d]) for d in D} for r in df.iloc if r['Label'] == o] for o in [0, 1]}
FCB = {o : F(CB[o], x) for o in [0, 1]}
GCB = {o : G(CB[o], x) for o in [0, 1]}
Φ = {s : FCB[s] for s in [0, 1]}

# Check that the cases are labeled correctly according to Psi, if it exists.
if P != None:
    strange = []
    print("Checking that the cases are labeled correctly.")
    cons = {o : 0 for o in [0, 1]}
    for o in [0, 1]:
        s.push()
        s.add(Not(P) if o == 0 else P)
        for a in tqdm(CB[o]):
            s.push()
            s.add(eq(a, x))
            if s.check() == sat:
                cons[o] += 1
            else:
                strange += [a]
            s.pop()
        s.pop()

    # Printing results.
    if strange == []:
        print("All cases are labeled correctly.")
    else:
        print("The following cases are labeled incorrectly:")
        for a in strange:
            print(a)

age: Ascending (2.74)
paid_contribution_1: [0] no (-0.35) < [1] yes (0.35)
paid_contribution_2: [0] no (-0.34) < [1] yes (0.34)
paid_contribution_3: [0] no (-0.46) < [1] yes (0.46)
paid_contribution_4: [0] no (-0.27) < [1] yes (0.27)
paid_contribution_5: [0] no (-0.56) < [1] yes (0.56)
is_spouse: [0] F (-1.8) < [1] T (1.8)
is_absent: [0] T (-1.76) < [1] F (1.76)
capital_resources: Descending (-2.63)
Checking that the cases are labeled correctly.


100%|██████████| 566/566 [00:00<00:00, 1560.82it/s]

All cases are labeled correctly.


Comparison
```
gender: [0] Male (-0.01) < [1] Female (0.01)
gender: [0] Male (-0.01) < [1] Female (0.01)
SeniorCitizen: [0] F (-0.15) < [1] T (0.15)
SeniorCitizen: [0] F (-0.04) < [1] T (0.04)
Partner: [0] Yes (-0.15) < [1] No (0.15)
Partner: [0] No (-0.0) < [1] Yes (0.0)
Dependents: [0] Yes (-0.16) < [1] No (0.16)
Dependents: [0] Yes (-0.03) < [1] No (0.03)
tenure: Descending (-0.35)
tenure: Descending (-1.42)
PhoneService: [0] No (-0.01) < [1] Yes (0.01)
PhoneService: [0] Yes (-0.01) < [1] No (0.01)
MultipleLines: [0] No (-0.03) < [1] No phone service (-0.01) < [2] Yes (0.04)
MultipleLines: [0] No (-0.09) < [1] No phone service (0.01) < [2] Yes (0.08)
InternetService: [0] No (-0.23) < [1] DSL (-0.12) < [2] Fiber optic (0.31)
InternetService: [0] DSL (-0.26) < [1] No (-0.08) < [2] Fiber optic (0.32)
OnlineSecurity: [0] No internet service (-0.23) < [1] Yes (-0.17) < [2] No (0.34)
OnlineSecurity: [0] No internet service (-0.08) < [1] Yes (-0.04) < [2] No (0.11)
OnlineBackup: [0] No internet service (-0.23) < [1] Yes (-0.08) < [2] No (0.27)
OnlineBackup: [0] No internet service (-0.08) < [1] Yes (0.02) < [2] No (0.05)
DeviceProtection: [0] No internet service (-0.23) < [1] Yes (-0.06) < [2] No (0.25)
DeviceProtection: [0] No internet service (-0.08) < [1] No (0.03) < [2] Yes (0.04)
TechSupport: [0] No internet service (-0.23) < [1] Yes (-0.16) < [2] No (0.34)
TechSupport: [0] No internet service (-0.08) < [1] Yes (-0.04) < [2] No (0.1)
StreamingTV: [0] No internet service (-0.23) < [1] Yes (0.07) < [2] No (0.13)
StreamingTV: [0] No internet service (-0.08) < [1] No (-0.06) < [2] Yes (0.13)
StreamingMovies: [0] No internet service (-0.23) < [1] Yes (0.06) < [2] No (0.13)
StreamingMovies: [0] No internet service (-0.08) < [1] No (-0.06) < [2] Yes (0.13)
Contract: [0] Two year (-0.3) < [1] One year (-0.18) < [2] Month-to-month (0.4)
Contract: [0] Two year (-0.32) < [1] One year (-0.03) < [2] Month-to-month (0.3)
PaperlessBilling: [0] No (-0.19) < [1] Yes (0.19)
PaperlessBilling: [0] No (-0.08) < [1] Yes (0.08)
PaymentMethod: [0] Credit card (automatic) (-0.13) < [1] Bank transfer (automatic) (-0.12) < [2] Mailed check (-0.09) < [3] Electronic check (0.3)
PaymentMethod: [0] Credit card (automatic) (-0.06) < [1] Mailed check (-0.05) < [2] Bank transfer (automatic) (-0.02) < [3] Electronic check (0.12)
MonthlyCharges: Ascending (0.19)
MonthlyCharges: Descending (-0.54)
TotalCharges: Descending (-0.2)
TotalCharges: Ascending (0.68)
```
Churn (Log. Reg.)
```
gender: [0] Male (-0.01) < [1] Female (0.01)
SeniorCitizen: [0] F (-0.04) < [1] T (0.04)
Partner: [0] No (-0.0) < [1] Yes (0.0)
Dependents: [0] Yes (-0.03) < [1] No (0.03)
tenure: Descending (-1.42)
PhoneService: [0] Yes (-0.01) < [1] No (0.01)
MultipleLines: [0] No (-0.09) < [1] No phone service (0.01) < [2] Yes (0.08)
InternetService: [0] DSL (-0.26) < [1] No (-0.08) < [2] Fiber optic (0.32)
OnlineSecurity: [0] No internet service (-0.08) < [1] Yes (-0.04) < [2] No (0.11)
OnlineBackup: [0] No internet service (-0.08) < [1] Yes (0.02) < [2] No (0.05)
DeviceProtection: [0] No internet service (-0.08) < [1] No (0.03) < [2] Yes (0.04)
TechSupport: [0] No internet service (-0.08) < [1] Yes (-0.04) < [2] No (0.1)
StreamingTV: [0] No internet service (-0.08) < [1] No (-0.06) < [2] Yes (0.13)
StreamingMovies: [0] No internet service (-0.08) < [1] No (-0.06) < [2] Yes (0.13)
Contract: [0] Two year (-0.32) < [1] One year (-0.03) < [2] Month-to-month (0.3)
PaperlessBilling: [0] No (-0.08) < [1] Yes (0.08)
PaymentMethod: [0] Credit card (automatic) (-0.06) < [1] Mailed check (-0.05) < [2] Bank transfer (automatic) (-0.02) < [3] Electronic check (0.12)
MonthlyCharges: Descending (-0.54)
TotalCharges: Ascending (0.68)
```

Churn (Pearson)
```
gender: [0] Male (-0.01) < [1] Female (0.01)
SeniorCitizen: [0] F (-0.15) < [1] T (0.15)
Partner: [0] Yes (-0.15) < [1] No (0.15)
Dependents: [0] Yes (-0.16) < [1] No (0.16)
tenure: Descending (-0.35)
PhoneService: [0] No (-0.01) < [1] Yes (0.01)
MultipleLines: [0] No (-0.03) < [1] No phone service (-0.01) < [2] Yes (0.04)
InternetService: [0] No (-0.23) < [1] DSL (-0.12) < [2] Fiber optic (0.31)
OnlineSecurity: [0] No internet service (-0.23) < [1] Yes (-0.17) < [2] No (0.34)
OnlineBackup: [0] No internet service (-0.23) < [1] Yes (-0.08) < [2] No (0.27)
DeviceProtection: [0] No internet service (-0.23) < [1] Yes (-0.06) < [2] No (0.25)
TechSupport: [0] No internet service (-0.23) < [1] Yes (-0.16) < [2] No (0.34)
StreamingTV: [0] No internet service (-0.23) < [1] Yes (0.07) < [2] No (0.13)
StreamingMovies: [0] No internet service (-0.23) < [1] Yes (0.06) < [2] No (0.13)
Contract: [0] Two year (-0.3) < [1] One year (-0.18) < [2] Month-to-month (0.4)
PaperlessBilling: [0] No (-0.19) < [1] Yes (0.19)
PaymentMethod: [0] Credit card (automatic) (-0.13) < [1] Bank transfer (automatic) (-0.12) < [2] Mailed check (-0.09) < [3] Electronic check (0.3)
MonthlyCharges: Ascending (0.19)
TotalCharges: Descending (-0.2)
```

Welfare-JAIR (Log. Reg.)
```
age: Ascending (3.2)
paid_contribution_1: [0] no (-0.58) < [1] yes (0.58)
paid_contribution_2: [0] no (-0.58) < [1] yes (0.58)
paid_contribution_3: [0] no (-0.57) < [1] yes (0.57)
paid_contribution_4: [0] no (-0.56) < [1] yes (0.56)
paid_contribution_5: [0] no (-0.58) < [1] yes (0.58)
is_spouse: [0] F (-2.99) < [1] T (2.99)
is_absent: [0] T (-3.01) < [1] F (3.01)
capital_resources: Descending (-3.8)
```

## Compute the landmarks

In [11]:
CB[0] = L[0]
CB[1] = L[1]
GCB = {o : G(CB[o], x) for o in [0, 1]}

In [10]:
print(len(L[0]))
print(len(L[1]))

202
559


In [12]:
# Initialize the landmarks.
L = {o : [] for o in [0, 1]}
for o in [0, 1]:
    s.push()
    s.add(GCB[o])
    for a in tqdm(CB[o]):
        s.push()
        s.add(eq(a, x))
        L[o] += [a] if s.check() == unsat else []
        s.pop()
    s.pop()

# Print the results.
print(f"Landmarks with outcome 0: {len(L[0])} / {len(CB[0])} = {len(L[0]) / len(CB[0])}")
print(f"Landmarks with outcome 1: {len(L[1])} / {len(CB[1])} = {len(L[1]) / len(CB[1])}")
print(f"Total number of landmarks: {len(L[0] + L[1])}")

# Make the lambda forcing formulas.
FL = {o : F(L[o], x) for o in [0, 1]}

# Update the forcing formulas to use the landmarks instead. 
Φ = {s : FL[s] for s in [0, 1]}

100%|██████████| 559/559 [00:00<00:00, 979.99it/s]

Landmarks with outcome 0: 14 / 202 = 0.06930693069306931
Landmarks with outcome 1: 5 / 559 = 0.008944543828264758
Total number of landmarks: 19


## Compute the consistency percentage

In [13]:
# Compute the number of inconsistent cases. 
cons = {o : 0 for o in [0, 1]}
total = len(CB[0] + CB[1])
for o in [0, 1]:
    s.push()
    s.add(Φ[1 - o])    
    for a in tqdm(CB[o]):
        s.push()
        s.add(eq(a, x))
        cons[o] += s.check() == unsat
        s.pop()
    s.pop()

# Print the results.
for o in [0, 1]:
    print(f"Consistency for outcome {o}:")
    print(f"{cons[o]} / {len(CB[o])} = {round(cons[o] / len(CB[o]), 3)}")

print("Total consistency:")
print(f"{cons[0] + cons[1]} / {len(CB[0] + CB[1])} = {round((cons[0] + cons[1]) / (len(CB[0] + CB[1])), 3)}")

100%|██████████| 559/559 [00:00<00:00, 1193.96it/s]

Consistency for outcome 0:
202 / 202 = 1.0
Consistency for outcome 1:
559 / 559 = 1.0
Total consistency:
761 / 761 = 1.0


In [8]:
# # Add a 100 y/o recidivist with 4 prior convictions to the case base. 
# CB[1] += [{'Age' : 100, 'Priors' : 4, 'Label' : True}]

# # Add the axiom that age cannot go above 100. 
# s.add(x['Age'] <= 100)

# # Re-compute the (strict) forcing formula for this case base. 
# GCB = {o : G(CB[o], x) for o in [0, 1]}

# # Re-initialize the landmarks.
# L = {o : [] for o in [0, 1]}
# for o in [0, 1]:
#     s.push()
#     s.add(GCB[o])
#     for a in tqdm(CB[o]):
#         s.push()
#         s.add(eq(a, x))
#         L[o] += [a] if s.check() == unsat else []
#         s.pop()
#     s.pop()

# print(f"Landmarks with outcome 0: {len(L[0])} / {len(CB[0])} = {len(L[0]) / len(CB[0])}")
# print(L[0])
# print(f"Landmarks with outcome 1: {len(L[1])} / {len(CB[1])} = {len(L[1]) / len(CB[1])}")
# print(L[1])

# # Re-compute the landmark forcing formulas.
# FL = {o : F(L[o], x) for o in [0, 1]}

## Soundness, completeness, and label equivalence checks. 

In [14]:
Results = []

# A list of counter_examples to the properties.
counter_examples = {}

# Initialize the properties to be checked.
checks = {
    "Φ₀ -> Φ₁" : Not(Implies(Φ[0], Φ[1])),
    "Φ₁ -> Φ₀" : Not(Implies(Φ[1], Φ[0])),
    "Consistency" : And(Φ[0], Φ[1]),
    "Completeness" : Not(Or(Φ[0], Φ[1])),
}
if P != None:
    checks |= {
        "Φ₀ -> ¬Ψ" : Not(Implies(Φ[0], Not(P))),
        "¬Ψ -> Φ₀" : Not(Implies(Not(P), Φ[0])),
        "Φ₁ -> Ψ" : Not(Implies(Φ[1], P)),
        "Ψ -> Φ₁" : Not(Implies(P, Φ[1])),
        "Φ₀ -> Ψ" : Not(Implies(Φ[0], P)),
        "Ψ -> Φ₀" : Not(Implies(P, Φ[0])),
        "Φ₁ -> ¬Ψ" : Not(Implies(Φ[1], Not(P))),
        "¬Ψ -> Φ₁" : Not(Implies(Not(P), Φ[1])),
    }

# Check the properties and collect counter examples. 
for c in checks:
    s.push()
    s.add(checks[c])
    if s.check() == unsat:
        r = "x"
    else:
        r = " "
        counter_examples[c] = s.model()
    Results += [[c, r]]
    s.pop()

# Show the results.
print(tabulate(Results))
for c in counter_examples:
    print(f"\nCounter example to: {c}")
    m = counter_examples[c]
    for d in D[:-1]:
        print(f"{d}: {m[x[d]]}")

------------  -
Φ₀ -> Φ₁
Φ₁ -> Φ₀
Consistency   x
Completeness  x
Φ₀ -> ¬Ψ      x
¬Ψ -> Φ₀      x
Φ₁ -> Ψ       x
Ψ -> Φ₁       x
Φ₀ -> Ψ
Ψ -> Φ₀
Φ₁ -> ¬Ψ
¬Ψ -> Φ₁
------------  -

Counter example to: Φ₀ -> Φ₁
age: 59
paid_contribution_1: 1
paid_contribution_2: 1
paid_contribution_3: 1
paid_contribution_4: 0
paid_contribution_5: 1
is_spouse: 1
is_absent: 1
capital_resources: 2999

Counter example to: Φ₁ -> Φ₀
age: 60
paid_contribution_1: 1
paid_contribution_2: 1
paid_contribution_3: 1
paid_contribution_4: 1
paid_contribution_5: 1
is_spouse: 1
is_absent: 1
capital_resources: 2999

Counter example to: Φ₀ -> Ψ
age: 60
paid_contribution_1: 0
paid_contribution_2: 0
paid_contribution_3: 0
paid_contribution_4: 0
paid_contribution_5: 0
is_spouse: 0
is_absent: 0
capital_resources: 2999

Counter example to: Ψ -> Φ₀
age: 60
paid_contribution_1: 0
paid_contribution_2: 1
paid_contribution_3: 1
paid_contribution_4: 1
paid_contribution_5: 1
is_spouse: 1
is_absent: 1
capital_resources: 2999

Counter e

### Attempt to make the case base complete (in addition to sound)

In [7]:
while True:
    compl = Not(Or(Φ[0], Φ[1]))
    s.push()
    s.add(compl)
    if s.check() == unsat:
        print("The case base was made consistent and complete.")
        s.pop()
        break
    else:
        m = s.model()
        a = {d : m[x[d]] for d in D[:-1]}
        print(a)
        s.pop()
        s.push()
        s.add(P)
        s.add(eq(a, x))
        o = 1 if s.check() == sat else 0
        a['Label'] = o
        L[o] += [a]
        s.pop()
        FL = {o : F(L[o], x) for o in [0, 1]}
        Φ = {s : FL[s] for s in [0, 1]}
        cons = And(Φ[0], Φ[1])
        s.push()
        s.add(cons)
        if s.check() != unsat:
            print("The case base has become inconsistent: quitting.")
            s.pop()
            break
        else:
            s.pop()
            continue

{'age': 95, 'paid_contribution_1': 1, 'paid_contribution_2': 1, 'paid_contribution_3': 1, 'paid_contribution_4': 1, 'paid_contribution_5': 1, 'is_spouse': 0, 'is_absent': 1, 'capital_resources': 48}
{'age': 95, 'paid_contribution_1': 1, 'paid_contribution_2': 1, 'paid_contribution_3': 1, 'paid_contribution_4': 1, 'paid_contribution_5': 1, 'is_spouse': 1, 'is_absent': 0, 'capital_resources': 48}
{'age': 95, 'paid_contribution_1': 0, 'paid_contribution_2': 1, 'paid_contribution_3': 0, 'paid_contribution_4': 1, 'paid_contribution_5': 0, 'is_spouse': 1, 'is_absent': 1, 'capital_resources': 47}
{'age': 95, 'paid_contribution_1': 1, 'paid_contribution_2': 0, 'paid_contribution_3': 1, 'paid_contribution_4': 0, 'paid_contribution_5': 0, 'is_spouse': 1, 'is_absent': 1, 'capital_resources': 46}
{'age': 96, 'paid_contribution_1': 0, 'paid_contribution_2': 0, 'paid_contribution_3': 1, 'paid_contribution_4': 1, 'paid_contribution_5': 1, 'is_spouse': 1, 'is_absent': 1, 'capital_resources': 157}
{'ag